# FPP GASC v1.7 benchmarking 

In [ ]:
using Revise
using FUSE
using IMAS
using Plots;
FUSE.logging(Logging.Info);

## Make new FPP equilibirum from scratch

### Initialize from FPP v1_demount

In [ ]:
dd = IMAS.dd();
ini, act = FUSE.case_parameters(:FPP, version=:v1_demount, init_from=:scalars);
#display(ini)

### Modify ini

In [ ]:
ini.equilibrium.R0 = 4.82
ini.equilibrium.B0 = 4.71
ini.equilibrium.ip = 9.01e6
ini.equilibrium.ϵ = 1/3.5
ini.equilibrium.κ = 2.19
ini.equilibrium.δ = 0.7
ini.equilibrium.ζ = 0.0

ini.build.plasma_gap = 0.2
ini.build.layers[:hfs_vacuum_vessel] += ini.build.layers[:hfs_vacuum_vessel_wall_inner] + ini.build.layers[:hfs_vacuum_vessel_wall_outer]
delete!(ini.build.layers,:hfs_vacuum_vessel_wall_inner)
delete!(ini.build.layers,:hfs_vacuum_vessel_wall_outer)
ini.build.layers[:lfs_vacuum_vessel] += ini.build.layers[:lfs_vacuum_vessel_wall_inner] + ini.build.layers[:lfs_vacuum_vessel_wall_outer]
delete!(ini.build.layers,:lfs_vacuum_vessel_wall_inner)
delete!(ini.build.layers,:lfs_vacuum_vessel_wall_outer)

ini.core_profiles.impurity = :Kr
ini.core_profiles.n_shaping = 1.5
ini.core_profiles.T_shaping = 1.5
ini.core_profiles.T_ratio = 1.0
ini.core_profiles.greenwald_fraction = 0.87
ini.core_profiles.ne_ped = 1.0e20
ini.core_profiles.greenwald_fraction_ped = missing

ini.equilibrium.boundary_from = :scalars
ini.equilibrium.xpoints_number = 1
ini.equilibrium.field_null_surface = 0.5

ini.ec_launchers.power_launched = 75.4e6

ini.pf_active.n_coils_outside = 8

ini.gasc.filename = missing
ini.gasc.case = missing

ini.requirements.flattop_duration = 24*3600

display(ini)

### Modify act

In [ ]:
act.ActorEquilibrium.model = :Solovev
act.ActorEquilibrium.symmetrize = true
act.ActorCHEASE.free_boundary = false
act.ActorCXbuild.rebuild_wall = true

act.ActorHFSsizing.do_plot = true
act.ActorHFSsizing.verbose = true
act.ActorHFSsizing.j_tolerance = 1.0
act.ActorHFSsizing.stress_tolerance = 0.0
act.ActorHFSsizing.aspect_ratio_tolerance = 0.0

act.ActorEquilibriumTransport.max_iter = 8
act.ActorEquilibriumTransport.convergence_error = 1e-2
act.ActorEquilibriumTransport.do_plot = true
act.ActorTauenn.transport_model = :ds03
act.ActorTauenn.confinement_factor = 1.0
act.ActorTauenn.T_shaping = 2.5
act.ActorTauenn.temp_pedestal_ratio = 1.0
act.ActorTauenn.error = 1e-3

act.ActorPFcoilsOpt.do_plot = true
act.ActorPFcoilsOpt.symmetric = true
act.ActorPFcoilsOpt.update_equilibrium = false
act.ActorPFcoilsOpt.optimization_scheme = :currents

act.ActorPowerNeeds.model = :thermal_power_fraction
act.ActorPowerNeeds.thermal_power_fraction = 0.1

act.ActorThermalCycle.power_cycle_type = :fixed_cycle_efficiency
act.ActorThermalCycle.fixed_cycle_efficiency = 0.44

act.ActorCosting.model = :Sheffield

display(act)

## Run WholeFacility Actor

In [ ]:
dd = IMAS.dd();
@time FUSE.init(dd, ini, act; do_plot=false);
#FUSE.digest(dd)

### Modify dd IMAS parameters

In [ ]:
dd.build.layer[8].material = "Steel, Stainless 316"
dd.build.layer[18].material = "Steel, Stainless 316"
dd.build.layer[24].material = "Steel, Stainless 316"
dd.build.tf.technology.temperature = 4.5
dd.build.oh.technology.temperature = 4.5

In [ ]:
@time FUSE.ActorWholeFacility(dd, act);

## Summary

In [ ]:
FUSE.digest(dd)

# Checks

## TF current density

In [ ]:
using Printf

R0 = dd.equilibrium.vacuum_toroidal_field.r0
B0 = dd.equilibrium.vacuum_toroidal_field.b0[1]
Itf = 5*B0*R0
Rtf_start = dd.build.layer[4].start_radius
Rtf_end = dd.build.layer[4].end_radius
Atf = pi*(Rtf_end^2-Rtf_start^2)
Jtf = Itf/Atf
Btf = 0.2*Itf/(Rtf_end)

fHTSinTapeTF = 1/46.54
ratioSCtoCopperTF = 1.0
fractionCableTF = (1-dd.build.tf.technology.fraction_stainless-dd.build.tf.technology.fraction_void)
fractionTapeTF = fractionCableTF * ratioSCtoCopperTF/(1+ratioSCtoCopperTF)
fractionSCTF = fHTSinTapeTF * fractionTapeTF

Jsc = Jtf/fractionSCTF

Jcrit_sc = FUSE.YBCO_Jcrit(Btf,0.0,dd.build.tf.technology.temperature)[1]/1e6

Jcrit_tf = Jcrit_sc * fractionSCTF

@printf("R_0 = %.2f m\n",R0)
@printf("B_0 = %.2f T\n",B0)
@printf("I_TF = %.2f MA\n", Itf)
@printf("J_TF = %.2f MA/m^2\n", Jtf)
@printf("B_TF = %.2f T\n", Btf)
@printf("fractionCableTF = %.4f \n", fractionCableTF)
@printf("fractionTapeTF = %.4f \n", fractionTapeTF)
@printf("fractionSCTF = %.4f \n", fractionSCTF)

@printf("J_sc = %.2f MA/m^2\n", Jsc)

@printf("Jcrit_sc = %.2f MA/m^2\n", Jcrit_sc)
@printf("Jcrit_tf = %.2f MA/m^2\n", Jcrit_tf)
@printf("Jmargin_tf = %.2f \n", (Jcrit_tf-Jtf)/Jtf)
@printf("Jmargin_sc = %.2f \n", (Jcrit_sc-Jsc)/Jsc)



## Power Balance

In [ ]:
display(dd.balance_of_plant)
display(dd.core_sources)

In [ ]:
powerAux = dd.ec_launchers.beam[1].power_launched.data[1]
powerAuxElectric = dd.balance_of_plant.power_electric_plant_operation.system[1].power[1]
powerBoP = dd.balance_of_plant.power_electric_plant_operation.system[2].power[1]

powerLine = dd.core_sources.source[8].profiles_1d[1].electrons.power_inside[end]
powerSync = dd.core_sources.source[4].profiles_1d[1].electrons.power_inside[end]
powerBrehm = dd.core_sources.source[3].profiles_1d[1].electrons.power_inside[end]
powerRad = powerLine + powerSync + powerBrehm

powerFusion = dd.summary.fusion.power.value[1]*5
powerNeutrons = powerFusion*0.8
powerAlphas = powerFusion*0.2
powerHeat = powerAlphas + powerAux
blanketMult = act.ActorBlanket.blanket_multiplier
powerRadFrac = powerRad/powerHeat

powerLFSBlanket = dd.blanket.module[1].time_slice[1].power_thermal_extracted
#powerHFSBlanket = dd.blanket.module[2].time_slice[1].power_thermal_extracted
powerBlanket = powerLFSBlanket #+ powerHFSBlanket
powerBoPBreeder = dd.balance_of_plant.heat_transfer.breeder.heat_load[1]
powerBoPWall = dd.balance_of_plant.heat_transfer.wall.heat_load[1]
powerBoPDivertors = dd.balance_of_plant.heat_transfer.divertor.heat_load[1]
powerDivertors = dd.divertors.divertor[1].power_thermal_extracted.data[1]
powerSOL = powerAlphas + powerAux + powerRad
fractionNeutronEnergyCaptured = powerBlanket / blanketMult / powerNeutrons

powerRunPlant = powerBoP + powerAuxElectric
powerThermalCycle = dd.balance_of_plant.thermal_cycle.total_useful_heat_power[1]
powerElectricGross = dd.balance_of_plant.thermal_cycle.power_electric_generated[1]
powerElectricNet = powerElectricGross - powerRunPlant

@printf("powerFusion = %.2f MW\n",powerFusion/1e6)
@printf("powerNeutrons = %.2f MW\n",powerNeutrons/1e6)
@printf("powerLFSBlanket = %.2f MW\n",powerLFSBlanket/1e6)
#@printf("powerHFSBlanket = %.2f MW\n",powerHFSBlanket/1e6)
@printf("powerBlanket = %.2f MW\n",powerBlanket/1e6)
@printf("powerBoPBreeder = %.2f MW\n",powerBoPBreeder/1e6)
@printf("fractionNeutronEnergyCaptured = %.3f \n",fractionNeutronEnergyCaptured)
@printf("\n")
@printf("powerLine = %.2f MW\n",powerLine/1e6)
@printf("powerSync = %.2f MW\n",powerSync/1e6)
@printf("powerBrehm = %.2f MW\n",powerBrehm/1e6)
@printf("powerRad = %.2f MW\n",powerRad/1e6)
@printf("powerBoPWall = %.2f MW\n",powerBoPWall/1e6)
@printf("\n")
@printf("powerAux = %.2f MW\n",powerAux/1e6)
@printf("powerAlphas = %.2f MW\n",powerAlphas/1e6)
@printf("powerHeat = %.2f MW\n",powerHeat/1e6)
@printf("RadFrac = %.3f\n",-1*powerRadFrac)
@printf("powerSOL = %.2f MW\n",powerSOL/1e6)
@printf("powerBoPDivertors = %.2f MW\n",powerBoPDivertors/1e6)
@printf("powerDivertors = %.2f MW\n",powerDivertors/1e6)
@printf("\n")
@printf("powerAuxElectric = %.2f MW\n",powerAuxElectric/1e6)
@printf("powerBoP = %.2f MW\n",powerBoP/1e6)
@printf("powerRunPlant = %.2f MW\n",powerRunPlant/1e6)
@printf("powerThermalCycle = %.2f MW\n",powerThermalCycle/1e6)
@printf("powerElectricGross = %.2f MW\n",powerElectricGross/1e6)
@printf("powerElectricNet = %.2f MW\n",powerElectricNet/1e6)



## Costing

In [ ]:
display(dd.costing)

In [ ]:
#display(dd.build.tf)

primary_coils_hfs = IMAS.get_build(dd.build, type = IMAS._tf_, fs = IMAS._hfs_)
display(primary_coils_hfs)

volumeTF = dd.build.layer[4].volume
stainlessFractionTF = dd.build.tf.technology.fraction_stainless
voidFractionTF = dd.build.tf.technology.fraction_void
ratioTF = dd.build.tf.technology.ratio_SC_to_copper
volumeSteelTF = volumeTF * stainlessFractionTF
volumeCopperTF = volumeTF*(1-stainlessFractionTF-voidFractionTF)*1/(1+ratioTF)
volumeSCTF = volumeTF*(1-stainlessFractionTF-voidFractionTF)*ratioTF/(1+ratioTF)

volumeOH = dd.build.layer[2].volume
stainlessFractionOH = dd.build.oh.technology.fraction_stainless
voidFractionOH = dd.build.oh.technology.fraction_void
ratioOH = dd.build.oh.technology.ratio_SC_to_copper
volumeSteelOH = volumeOH * stainlessFractionOH
volumeCopperOH = volumeOH*(1-stainlessFractionOH-voidFractionOH)*1/(1+ratioOH)
volumeSCOH = volumeOH*(1-stainlessFractionOH-voidFractionOH)*ratioOH/(1+ratioOH)

production_increase = dd.costing.future.learning.hts.production_increase
learning_rate = dd.costing.future.learning.hts.learning_rate
hts_discount_factor = production_increase^(log(learning_rate) / log(2))

costSteelTF = volumeSteelTF * 0.36
costCopperTF = volumeCopperTF * 0.5
costSCTF = volumeSCTF * (87.5/2) * hts_discount_factor

costSteelOH = volumeSteelOH * 0.36
costCopperOH = volumeCopperOH * 0.5
costSCOH = volumeSCOH * (87.5/2) * hts_discount_factor

costTF = 1.5*(costSteelTF+costCopperTF+costSCTF)
costTFtoday = costTF * 300/240
costPrimaryCoils = dd.costing.cost_direct_capital.system[2].subsystem[2].cost 

@printf("volumeTF = %.2f m^3\n", volumeTF)
@printf("volumeSCTF = %.2f m^3\n", volumeSCTF)
@printf("stainlessFractionTF = %.3f\n", stainlessFractionTF)
@printf("volumeOH = %.2f m^3\n", volumeOH)
@printf("volumeSCOH = %.2f m^3\n", volumeSCOH)
@printf("stainlessFractionOH = %.3f\n", stainlessFractionOH)
@printf("\n")
@printf("prodution_increase = %.2f\n",production_increase)
@printf("learning_rate = %.2f\n", learning_rate)
@printf("hts_discount_factor = %.2f\n", hts_discount_factor)
@printf("\n")
@printf("costSteelTF = %.2f \$M\n",costSteelTF)
@printf("costCopperTF = %.2f \$M\n",costCopperTF)
@printf("costSCTF = %.2f \$M\n",costSCTF)
@printf("costSteelOH = %.2f \$M\n",costSteelOH)
@printf("costCopperOH = %.2f \$M\n",costCopperOH)
@printf("costSCOH = %.2f \$M\n",costSCOH)
@printf("costTF = %.2f \$M\n", costTF)
@printf("costTFtoday = %.2f \$M\n", costTFtoday)
@printf("costPrimaryCoils = %.2f \$M\n", costPrimaryCoils)

## Confinement scaling factor